In [25]:

import gzip
import math
import numpy as  np
import random
import sklearn
import string
from collections import defaultdict
import json
def parseData(file):
    for l in open(file,'r'):
        yield json.loads(l)

In [37]:
dataset = []
punctuation = set(string.punctuation)
f = gzip.open("modcloth_final_data.json.gz")
for l in f:
    d = eval(l)
    dataset.append(d)

In [38]:
fittingPerUser = defaultdict(list)
fittingPerItem = defaultdict(list)

for entry in dataset:
    item_id = entry['item_id']
    user_id = entry['user_id']
    fit = entry['fit']
    fittingPerUser[user_id].append((item_id,fit))
    fittingPerItem[item_id].append((user_id,fit))

print('Size of the dataset :', len(dataset))
print('Size of the user base: ', len(fittingPerUser))
print('Size of the item base: ', len(fittingPerItem))

Size of the dataset : 82790
Size of the user base:  47958
Size of the item base:  1378


In [48]:
lengths = []
for u in fittingPerUser:
    lengths.append(len(fittingPerUser[u]))
print("average fitting per user :", np.array(lengths).mean())
print('number of unique users: ', len(lengths))
lengths = []
for i in fittingPerItem:
    lengths.append(len(fittingPerItem[i]))

print("average fitting per item :", np.array(lengths).mean())
print('number of unique items: ', len(lengths))

print('-----------------------------')
print("Remove users with too few reviews")
print('-----------------------------')

lengths = []
for u in fittingPerUser:
    if len(fittingPerUser[u])>2:
        lengths.append(len(fittingPerUser[u]))
print("average fitting per user post filterng:", np.array(lengths).mean())
print("total number of users post filtering :", len(lengths))


average fitting per user : 1.7263021810751074
number of unique users:  47958
average fitting per item : 60.07982583454282
number of unique items:  1378
-----------------------------
Remove users with too few reviews
-----------------------------
average fitting per user post filterng: 4.525805147554238
total number of users post filtering : 7421


In [59]:


dataset2 = []
dataset2 = list(parseData('renttherunway_final_data.json'))

dataset2[0]

{'fit': 'fit',
 'user_id': '420272',
 'bust size': '34d',
 'item_id': '2260466',
 'weight': '137lbs',
 'rating': '10',
 'rented for': 'vacation',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'body type': 'hourglass',
 'review_summary': 'So many compliments!',
 'category': 'romper',
 'height': '5\' 8"',
 'size': 14,
 'age': '28',
 'review_date': 'April 20, 2016'}

In [60]:
bt = set()
for i in dataset2:
    
    bt.add(i['body type'])
    

KeyError: 'body type'

In [62]:
fittingPerUser = defaultdict(list)
fittingPerItem = defaultdict(list)

for entry in dataset2:
    item_id = entry['item_id']
    user_id = entry['user_id']
    fit = entry['fit']
    fittingPerUser[user_id].append((item_id,fit))
    fittingPerItem[item_id].append((user_id,fit))

print('Size of the dataset :', len(dataset))
print('Size of the user base: ', len(fittingPerUser))
print('Size of the item base: ', len(fittingPerItem))

Size of the dataset : 82790
Size of the user base:  105571
Size of the item base:  5850


In [63]:
lengths = []
for u in fittingPerUser:
    lengths.append(len(fittingPerUser[u]))
print("average fitting per user :", np.array(lengths).mean())
lengths = []
for i in fittingPerItem:
    lengths.append(len(fittingPerItem[i]))

print("average fitting per item :", np.array(lengths).mean())



average fitting per user : 1.8238341968911918
average fitting per item : 32.91350427350427


In [66]:
lengths = []
total_reviews = 0
for u in fittingPerUser:
    if len(fittingPerUser[u])>2:
        lengths.append(len(fittingPerUser[u]))
        total_reviews+= len(fittingPerUser[u])
print("average fitting per user :", np.array(lengths).mean())
print(len(lengths))
print(total_reviews)
lengths = []
for i in fittingPerItem:
    lengths.append(len(fittingPerItem[i]))

print("average fitting per item :", np.array(lengths).mean())

average fitting per user : 5.3576835730507195
15852
84930
average fitting per item : 32.91350427350427


In [33]:
len(lengths)

1378